# Machne Learning Multi-step forecasts

Forecasting 7 days ahead using half hourly dataset

In [20]:
import sys
from math import sqrt
from numpy import split
from numpy import array
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import SGDRegressor

sys.path.append("../scripts/")
from forecast_utils import av_scores



In [21]:
PATH='../input/merged_data/'

In [22]:
DAILY_SAMPLE_RATE=48

In [23]:
# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into weeks
    #use last 8 weeks for test (enough for validation and test)
    week_hh = 7*DAILY_SAMPLE_RATE
    test_days = 8*week_hh
    train, test = data[:-(test_days)], data[test_days:]
    print('train: {0}, split weeks: {1}'.format(len(train), len(train)/week_hh))
    print('test: {0}, split weeks: {1}'.format(len(test), len(test)/week_hh))
    # restructure into windows of weekly data
    train = array(split(train, len(train)/week_hh))
    test = array(split(test, len(test)/week_hh))
    return train, test

In [24]:
# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        #print('pred: {0}'.format(predicted[0].shape))
        #print('act: {0}'.format(actual[0].shape))
        # calculate mse
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        # calculate rmse
        rmse = sqrt(mse)
        # store
        scores.append(rmse)
    # calculate overall RMSE
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
    return score, scores

In [25]:
# summarize scores
def print_scores(name, score, scores):
    #s_scores = ', '.join(['%.1f' % s for s in scores])
    #print('%s: [%.3f] %s' % ('Half hourly', score, s_scores))
    n_chunks = len(scores)/DAILY_SAMPLE_RATE
    print(type(scores))
    scores_chunked = np.array_split(scores, n_chunks)
    av_scores = []
    for chunk in scores_chunked:
        av_scores.append(np.average(chunk))
    w_scores = ', '.join(['%.1f' % s for s in av_scores])
    print('%s: [%.3f] %s' % (name, score, w_scores))


In [26]:
# summarize scores
def print_best_algo(name_score):
    best_alg = ''
    best_score = 0
    for i, n_s in enumerate(name_score):
        if i == 0:
            best_alg=n_s[0]
            best_score=n_s[1]
        else:
            if n_s[1]<best_score:
                best_alg=n_s[0]
                best_score=n_s[1]
    print('Best overall algorithm: {0}'.format(best_alg))

In [27]:
# prepare a list of ml models
def get_models(models=dict()):
    # linear models
    models['lr'] = LinearRegression()
    models['lasso'] = Lasso()
    models['ridge'] = Ridge()
    models['en'] = ElasticNet()
    models['huber'] = HuberRegressor()
    models['lars'] = Lars()
    models['llars'] = LassoLars()
    models['pa'] = PassiveAggressiveRegressor(max_iter=1000, tol=1e-3)
    models['ranscac'] = RANSACRegressor()
    models['sgd'] = SGDRegressor(max_iter=1000, tol=1e-3)
    print('Defined %d models' % len(models))
    return models

In [28]:
# create a feature preparation pipeline for a model
def make_pipeline(model):
    steps = list()
    # standardization
    steps.append(('standardize', StandardScaler()))
    # normalization
    steps.append(('normalize', MinMaxScaler()))
    # the model
    steps.append(('model', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

In [29]:
# convert history into inputs and outputs
def to_supervised(history, output_ix):
    X, y = list(), list()
    # step over the entire history one time step at a time
    for i in range(len(history)-1):
        X.append(history[i][:,0])
        y.append(history[i + 1][output_ix,0])
    return array(X), array(y)

In [30]:
# fit a model and make a forecast
def sklearn_predict(model, history):
    #print('len: {0}, history[0].shape: {1}'.format(len(history), history[0].shape))
    yhat_sequence = list()
    # fit a model for each forecast hh
    for i in range(7*DAILY_SAMPLE_RATE):
        # prepare data
        train_x, train_y = to_supervised(history, i)
        # make pipeline
        pipeline = make_pipeline(model)
        # fit the model
        pipeline.fit(train_x, train_y)
        # forecast
        x_input = array(train_x[-1, :]).reshape(1,7*DAILY_SAMPLE_RATE)
        yhat = pipeline.predict(x_input)[0]
        # store
        yhat_sequence.append(yhat)
    return yhat_sequence

In [31]:
# evaluate a single model
def evaluate_model(model, train, test):
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
        yhat_sequence = sklearn_predict(model, history)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
        history.append(test[i, :])
    predictions = array(predictions)
    # evaluate predictions days for each week
    score, scores = evaluate_forecasts(test[:, :, 0], predictions)
    return score, scores

### Read in data

In [32]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [33]:
dataset = pd.read_csv('{0}LCLid/clean_mac000230.csv'.format(PATH), parse_dates=['day_time'], date_parser=dateparse)
dataset.set_index(['day_time'],inplace=True)

In [34]:
dataset =  dataset[['energy(kWh/hh)', 'temperature', 'humidity']]
dataset=dataset.fillna(0)

In [35]:
# split into train and test
train, test = split_dataset(dataset.values)
# prepare the models to evaluate
models = get_models()

train: 36288, split weeks: 108.0
test: 36288, split weeks: 108.0
Defined 10 models


In [ ]:
# evaluate each model
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
score_list = []
for name, model in models.items():
    # evaluate and get scores
    score, scores = evaluate_model(model, train, test)
    score_list.append((name, score))
    # summarize scores
    print_scores(name, score, scores)
    av = av_scores(scores, DAILY_SAMPLE_RATE)
    # plot scores
    plt.scatter(days, scores, marker='o', label=name)
# show plot
plt.legend()
plt.show()

In [ ]:
best_alg = print_best_algo(score_list)

## Plots

Display history and forecasts on same plot

## Daily data

Run models on daily dataset